# Swin Transformer

## Preparation


In [ ]:
!apt install git curl patch
!pip install torch torchvision
!pip install timm==0.3.2

In [ ]:
!apt install ninja
!git clone https://github.com/NVIDIA/apex
# Disable CUDA version check for NVIDIA/apex since Colab has 11.0 while NVIDIA/apex expects 10.2
!sed -i "s@if (bare_metal_major != torch_binary_major) or (bare_metal_minor != torch_binary_minor):@if False:@" apex/setup.py
!cd apex && pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
!rm -r apex
!pip install opencv-python==4.4.0.46 termcolor==1.1.0 yacs==0.1.8

In [ ]:
%%bash
# Source code
rm -rf Swin-Transformer/
git clone https://github.com/microsoft/Swin-Transformer.git
cd Swin-Transformer/
mkdir -p output/{MNIST,CIFAR10,CIFAR100}
# Support MNIST and CIFAR-10 datasets
curl https://github.com/eadwu/Swin-Transformer/commit/5370644d8fe95fda32d56231a36613587029ab05.patch |
    patch -p1

In [ ]:
!pip install -U PyYAML

## Training (CIFAR-100)

In [ ]:
%%bash
# Swin-T
cd Swin-Transformer/
python -m torch.distributed.launch --nproc_per_node 1 --master_port 12345 main.py \
    --cfg configs/swin_tiny_patch4_window7_224.yaml --batch-size 128 --output output/CIFAR100 \
    --accumulation-steps 2 --cache-mode part \
    --opts SAVE_FREQ 10 PRINT_FREQ 1 TRAIN.EPOCHS 30 DATA.DATASET cifar100 &>> output/CIFAR100/Swin-T.log

In [ ]:
%%bash
zip -q -9 -r SW_output.zip Swin-Transformer/output/